# tf-ProtoNN 
### Implementation of ProtoNN in tensorflow (single-GPU version) for large-scale multilabel learning

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import numpy as np
import scipy.io as sio

from model import *
from preprocess import *

from trainer.single_gpu_train import Trainer
import cfgs.config_eurlex_with_preprocess as config

# Set GPU = 1
config.cfg.num_gpus = 1

### Display dataset and parameter stats

In [2]:
path = os.path.join(config.cfg.dir_name, "train_data")
X, Y = data_loader.data_loader(path, config.cfg)
print("Data-Stats:")
print("Num instances = ", X.shape[0])
print("Feature dimensionality = ", X.shape[-1])
print("Label dimensionality = ", Y.shape[-1])
print("Mean pts per label = ", Y.nnz/Y.shape[1])
print("Mean labels per pt = ", Y.nnz/Y.shape[0])

print("Param-stats:")
print("Projection-Dim: %d"%config.cfg.d)
print("Num Projections: %d"%config.cfg.m)

Data-Stats:
Num instances =  15539
Feature dimensionality =  5000
Label dimensionality =  3993
Mean pts per label =  20.6686701728
Mean labels per pt =  5.31115258382
Param-stats:
Projection-Dim: 250
Num Projections: 1000


### Preprocessing (PCA and clustering) for parameter initialization

In [ ]:
tic = time.time()
D = X.shape[-1]
d = config.cfg.d
m = config.cfg.m
W0, Wx = pca.train_pca(X, d)
B0 = clustering.train_kmeans(Wx, m, ngpu = 1).T
Z0 = prototypes.get_prototypes(Y, Wx, B0, num_pts_per_cluster=config.cfg.num_pts_per_cluster)
t_elapsed = time.time() - tic;
print("Time-taken for pre-training: %.4f"%(t_elapsed))

path = os.path.join(config.cfg.dir_name, "init_params_faiss.mat")
sio.savemat(path, {'W':W0, 'B':B0, 'Z':Z0})

Time-taken for pre-training: 72.6670


### Run training

In [ ]:
m = Trainer(config.cfg).train()

W variable created on gpu
B variable created on gpu
Z variable created on gpu
